In [ ]:
import ee
import geemap
import os
from datetime import datetime

# Trigger the authentication flow
ee.Authenticate()

# Initialize the library
ee.Initialize(project='btech-project-443813')  # Make sure this project ID is correct

# Define region of interest for Delhi
roi = ee.Geometry.Polygon([
    [76.8, 28.9],  # Northwest point
    [77.6, 28.9],  # Northeast point
    [77.6, 28.4],  # Southeast point
    [76.8, 28.4]   # Southwest point
])

# Define the function to calculate monthly cloud frequency
def get_monthly_cloud_frequency(year, month, region):
    # Use updated MODIS Collection 061 datasets
    start_date = f"{year}-{month:02d}-01"
    end_date = f"{year}-{month:02d}-{(datetime(year, month % 12 + 1, 1) - datetime(year, month, 1)).days:02d}"
    
    modis_terra = ee.ImageCollection("MODIS/061/MOD09GA").filterDate(
        start_date, end_date
    ).filterBounds(region)
    
    modis_aqua = ee.ImageCollection("MODIS/061/MYD09GA").filterDate(
        start_date, end_date
    ).filterBounds(region)
    
    # Print diagnostic information
    print(f"Terra images for {start_date} to {end_date}: {modis_terra.size().getInfo()}")
    print(f"Aqua images for {start_date} to {end_date}: {modis_aqua.size().getInfo()}")
    
    # Combine Terra and Aqua datasets
    combined = modis_terra.merge(modis_aqua)
    
    # Extract cloud flags from state_1km band (bit 10)
    # Use a more robust cloud masking approach
    def mask_clouds(img):
        # Cloud mask for MODIS (state_1km band)
        cloud_mask = img.select("state_1km").bitwiseAnd(1 << 10).eq(0)
        return img.updateMask(cloud_mask)
    
    # Apply cloud masking and filter out masked images
    cloud_masked = combined.map(mask_clouds)
    
    # Calculate cloud frequency
    cloud_frequency = ee.Image.constant(cloud_masked.size().divide(combined.size()).multiply(100)).clip(region)
    cloud_frequency = cloud_frequency.rename(f"cloud_freq_{year}_{month:02d}")
    
    print(f"Images after cloud masking for {start_date} to {end_date}: {cloud_masked.size().getInfo()}")
    
    return cloud_frequency

# Specify date range
start_year = 2000
end_year = 2022

# Output directory
output_dir = "/home/stormej/dev/varsha/data/cloud_cover/cloud_cover_tif"
os.makedirs(output_dir, exist_ok=True)

# Download monthly data
for year in range(start_year, end_year + 1):
    for month in range(1, 13):
        try:
            cloud_freq = get_monthly_cloud_frequency(year, month, roi)
            filename = os.path.join(output_dir, f"delhi_cloud_freq_{year}_{month:02d}.tif")
            
            # Export to GeoTIFF
            task = geemap.ee_export_image(
                cloud_freq,
                filename=filename,
                scale=1000,  # 1 km resolution
                region=roi,
                file_per_band=False,
            )
            print(f"Exporting {filename}")
        
        except Exception as e:
            print(f"Error processing year {year}, month {month}: {e}")

print("Cloud frequency data export for Delhi completed.")

Terra images for 2000-01-01 to 2000-01-31: 0
Aqua images for 2000-01-01 to 2000-01-31: 0
Images after cloud masking for 2000-01-01 to 2000-01-31: 0
Generating URL ...
Please wait ...
Data downloaded to /home/stormej/dev/varsha/data/cloud_cover/delhi_cloud_freq_2000_01.tif
Exporting /home/stormej/dev/varsha/data/cloud_cover/delhi_cloud_freq_2000_01.tif
Terra images for 2000-02-01 to 2000-02-29: 5
Aqua images for 2000-02-01 to 2000-02-29: 0
Images after cloud masking for 2000-02-01 to 2000-02-29: 5
Generating URL ...
Please wait ...
Data downloaded to /home/stormej/dev/varsha/data/cloud_cover/delhi_cloud_freq_2000_02.tif
Exporting /home/stormej/dev/varsha/data/cloud_cover/delhi_cloud_freq_2000_02.tif
Terra images for 2000-03-01 to 2000-03-31: 30
Aqua images for 2000-03-01 to 2000-03-31: 0
Images after cloud masking for 2000-03-01 to 2000-03-31: 30
Generating URL ...
Please wait ...
Data downloaded to /home/stormej/dev/varsha/data/cloud_cover/delhi_cloud_freq_2000_03.tif
Exporting /home/s

In [20]:
#Resampling lst monthly data to imd data i.e 0.25 degree

import os
import rasterio
from rasterio.enums import Resampling
# # Paths
# rainfall_folder = r'/home/stormej/dev/btech-project/data/rain_tif'  # Folder containing all rainfall files
rainfall_folder = r'/home/stormej/dev/varsha/data/reference_tif'  # Folder containing reference files
cloud_cover_folder = r'/home/stormej/dev/varsha/data/cloud_cover/cloud_cover_tif'  # Folder containing all LST files
output_folder = r'/home/stormej/dev/varsha/data/cloud_cover/cloud_cover_resampled'  # Folder to save resampled files

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


# Find a reference rainfall file
reference_rainfall_path = None
for filename in os.listdir(rainfall_folder):
    if filename.endswith('.tif'):
        reference_rainfall_path = os.path.join(rainfall_folder, filename)
        break

if reference_rainfall_path is None:
    raise FileNotFoundError("No TIF file found in the rainfall folder")

print(f"Using reference rainfall file: {reference_rainfall_path}")

# Open the reference rainfall file to get its properties
with rasterio.open(reference_rainfall_path) as rainfall_src:
    rainfall_shape = (rainfall_src.height, rainfall_src.width)
    rainfall_crs = rainfall_src.crs
    rainfall_transform = rainfall_src.transform

print(f"Reference rainfall data shape: {rainfall_shape}, CRS: {rainfall_crs}")

# Process all LST files in the folder
for filename in os.listdir(cloud_cover_folder):
    if filename.endswith('.tif'):
        lst_path = os.path.join(cloud_cover_folder, filename)
        print(f"\nProcessing: {filename}")
        
        with rasterio.open(lst_path) as lst_src:
            lst_data = lst_src.read(1)
            print(f"LST data shape: {lst_data.shape}, dtype: {lst_data.dtype}")
            
            # Resample LST data using average method
            resampled_lst = lst_src.read(
                out_shape=(1, rainfall_shape[0], rainfall_shape[1]),
                resampling=Resampling.average
            )
            print(f"Resampled LST shape: {resampled_lst.shape}, dtype: {resampled_lst.dtype}")
            
            # Extract date from LST filename (handling format: lst_south_2010-01-01.tif)
            # Split by underscore and get the last part before .tif
            # Extract date from filename (handling format: delhi_cloud_freq_2000_01.tif)
            # Split by underscore and get the last two parts joined by an underscore
            date_part = '_'.join(filename.split('_')[-2:]).split('.')[0]
            
            # Create output filename
            output_filename = f'modis_cloud_cover_avg_resampled_{date_part}.tif'
            output_path = os.path.join(output_folder, output_filename)
            
            # Save the resampled LST
            with rasterio.open(
                output_path,
                'w',
                driver='GTiff',
                height=resampled_lst.shape[1],
                width=resampled_lst.shape[2],
                count=1,
                dtype=resampled_lst.dtype,
                crs=rainfall_crs,
                transform=rainfall_transform,
            ) as dst:
                dst.write(resampled_lst[0], 1)
        
        print(f"Resampled Cloud Cover saved to: {output_path}")

print("\nAll Cloud Cover files have been resampled.")

Using reference rainfall file: /home/stormej/dev/varsha/data/reference_tif/refrence_tif.tif
Reference rainfall data shape: (129, 135), CRS: EPSG:4326

Processing: delhi_cloud_freq_2000_01.tif
LST data shape: (57, 90), dtype: float64
Resampled LST shape: (1, 129, 135), dtype: float64
Resampled Cloud Cover saved to: /home/stormej/dev/varsha/data/cloud_cover/cloud_cover_resampled/modis_cloud_cover_avg_resampled_2000_01.tif

Processing: delhi_cloud_freq_2000_02.tif
LST data shape: (57, 90), dtype: float64
Resampled LST shape: (1, 129, 135), dtype: float64
Resampled Cloud Cover saved to: /home/stormej/dev/varsha/data/cloud_cover/cloud_cover_resampled/modis_cloud_cover_avg_resampled_2000_02.tif

Processing: delhi_cloud_freq_2000_03.tif
LST data shape: (57, 90), dtype: float64
Resampled LST shape: (1, 129, 135), dtype: float64
Resampled Cloud Cover saved to: /home/stormej/dev/varsha/data/cloud_cover/cloud_cover_resampled/modis_cloud_cover_avg_resampled_2000_03.tif

Processing: delhi_cloud_fre